# Description

This notebook contains an example of accessing order book data from CCXT.

In [57]:
import logging
import pprint

import pandas as pd

import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import im_v2.ccxt.data.extract.extractor as imvcdexex
import im_v2.common.universe.universe as imvcounun
import im_v2.crypto_chassis.data.extract.extractor as imvccdexex

In [59]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO  # Git
  branch_name='CMTask2700_order_book_data'
  hash='fc44b1f02'
  # Last commits:
    * fc44b1f02 Nina Lee Cm task2682 update latest snapshot (#2716)                        (   4 hours ago) Fri Sep 2 12:06:32 2022  (HEAD -> CMTask2700_order_book_data, origin/master, origin/HEAD, origin/CMTask2700_order_book_data, master)
    * d1ea8623f DanilYachmenev Cm task2730 add fast test (#2758)                                 (  20 hours ago) Thu Sep 1 19:59:58 2022  (origin/CmTask2731_test_get_fills_CcxtBroker)
    * 9fc9bb1b4 Grigorii Pomazkin Cm task2755 create a testnet universe (#2757)                     (  21 hours ago) Thu Sep 1 18:54:27 2022           
# Machine info
  system=Linux
  node name=a13da9ed9824
  release=5.15.0-1019-aws
  version=#23~20.04.1-Ubuntu SMP Thu Aug 18 03:20:14 UTC 2022
  machine=x86_64
  processor=x86_64
  cpu count=4
  cpu freq=scpufreq(current=2499.998, min=0.0, max=0.0)
  memory=svmem(total=16583786496, available=1881284608, percent=88.7, used=139771

### Initialize extractors

In [29]:
# Initialize CCXT extractor.
ccxt_extractor = imvcdexex.CcxtExtractor("binance", "futures")
exchange = ccxt_extractor._exchange

In [33]:
# Initialize CryptoChassis extractor and download a bid/ask data example.
cryptochassis_extractor = imvccdexex.CryptoChassisExtractor("futures")
exchange_id = "binance"
currency_pair = "BTC_USDT"
start_timestamp = pd.Timestamp("01-09-2022")
end_timestamp = pd.Timestamp("02-09-2022")
cryptochassis_data_example = cryptochassis_extractor._download_bid_ask(
    exchange_id, currency_pair, start_timestamp, end_timestamp
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [01:01<00:00,  1.93s/it]


In [35]:
cryptochassis_data_example

,timestamp,bid_price,bid_size,ask_price,ask_size
0,1641686400,41688.05,2.582,41688.06,0.040
1,1641686401,41688.05,0.833,41688.06,0.049
2,1641686403,41688.05,1.457,41688.06,2.210
3,1641686404,41688.05,3.501,41688.06,1.087
4,1641686408,41688.05,1.036,41688.06,0.182
...,...,...,...,...,...
86375,1644451195,44354.70,2.630,44354.71,0.653
86376,1644451196,44358.08,0.123,44358.09,0.836
86377,1644451197,44358.08,5.082,44358.09,1.100
86378,1644451198,44358.08,5.078,44358.09,0.283


In [36]:
bid_ask_columns = cryptochassis_data_example.columns.to_list()

## Example of CCXT orderbook data

In [4]:
symbol = "BTC/USDT"
raw_orderbook_data = exchange.fetch_order_book(symbol)

In [14]:
# Get data keys from CCXT.
print(raw_orderbook_data.keys())

dict_keys(['symbol', 'bids', 'asks', 'timestamp', 'datetime', 'nonce'])


In [11]:
# Example of raw orderbook data.
pprint.pprint(raw_orderbook_data, sort_dicts=False)

{'symbol': 'BTC/USDT',
 'bids': [[20303.2, 26.543],
          [20303.1, 1.82],
          [20303.0, 2.74],
          [20302.8, 0.012],
          [20302.7, 0.262],
          [20302.6, 11.522],
          [20302.5, 3.246],
          [20302.0, 0.006],
          [20301.9, 0.002],
          [20301.8, 0.072],
          [20301.7, 0.178],
          [20301.6, 0.167],
          [20301.5, 0.038],
          [20301.4, 0.789],
          [20301.3, 0.02],
          [20301.2, 0.016],
          [20301.1, 0.619],
          [20301.0, 4.175],
          [20300.9, 0.907],
          [20300.8, 7.977],
          [20300.7, 4.337],
          [20300.6, 0.087],
          [20300.5, 3.551],
          [20300.4, 0.406],
          [20300.2, 0.53],
          [20300.1, 0.239],
          [20300.0, 0.141],
          [20299.9, 0.166],
          [20299.8, 1.549],
          [20299.7, 0.296],
          [20299.6, 1.163],
          [20299.5, 0.958],
          [20299.4, 4.116],
          [20299.3, 1.905],
          [20299.2, 0.006],

In [19]:
print(len(raw_orderbook_data["bids"]))

500


In [20]:
print(len(raw_orderbook_data["asks"]))

500


- The `fetch_order_book` method returns the 500 top of the book results at the moment of request.
- Note that the `timestamp` and `datetime` are set to a `ms` accuracy.

In [22]:
# Transform the raw data into a dataframe.
orderbook_df = pd.DataFrame.from_dict(raw_orderbook_data)
orderbook_df

,symbol,bids,asks,timestamp,datetime,nonce
0,BTC/USDT,"[20303.2, 26.543]","[20303.3, 0.01]",1662128478476,2022-09-02T14:21:18.476Z,1879833726652
1,BTC/USDT,"[20303.1, 1.82]","[20303.4, 0.001]",1662128478476,2022-09-02T14:21:18.476Z,1879833726652
2,BTC/USDT,"[20303.0, 2.74]","[20303.6, 0.047]",1662128478476,2022-09-02T14:21:18.476Z,1879833726652
3,BTC/USDT,"[20302.8, 0.012]","[20303.8, 0.002]",1662128478476,2022-09-02T14:21:18.476Z,1879833726652
4,BTC/USDT,"[20302.7, 0.262]","[20304.6, 1.328]",1662128478476,2022-09-02T14:21:18.476Z,1879833726652
...,...,...,...,...,...,...
495,BTC/USDT,"[20246.1, 0.394]","[20358.3, 0.911]",1662128478476,2022-09-02T14:21:18.476Z,1879833726652
496,BTC/USDT,"[20246.0, 0.645]","[20358.4, 0.213]",1662128478476,2022-09-02T14:21:18.476Z,1879833726652
497,BTC/USDT,"[20245.8, 0.246]","[20358.5, 0.055]",1662128478476,2022-09-02T14:21:18.476Z,1879833726652
498,BTC/USDT,"[20245.7, 0.031]","[20358.6, 6.3]",1662128478476,2022-09-02T14:21:18.476Z,1879833726652


Each row in the orderbook contains a `list` with 2 values corresponding to `bid_price`, `bid_size`, `ask_price` and `ask_size`, as in CryptoChassis.

In [25]:
orderbook_df[["bid_price", "bid_size"]] = pd.DataFrame(
    orderbook_df.bids.to_list(), index=orderbook_df.index
)
orderbook_df[["ask_price", "ask_size"]] = pd.DataFrame(
    orderbook_df.asks.to_list(), index=orderbook_df.index
)

In [26]:
orderbook_df

,symbol,bids,asks,timestamp,datetime,nonce,bid_price,bid_size,ask_price,ask_size
0,BTC/USDT,"[20303.2, 26.543]","[20303.3, 0.01]",1662128478476,2022-09-02T14:21:18.476Z,1879833726652,20303.2,26.543,20303.3,0.010
1,BTC/USDT,"[20303.1, 1.82]","[20303.4, 0.001]",1662128478476,2022-09-02T14:21:18.476Z,1879833726652,20303.1,1.820,20303.4,0.001
2,BTC/USDT,"[20303.0, 2.74]","[20303.6, 0.047]",1662128478476,2022-09-02T14:21:18.476Z,1879833726652,20303.0,2.740,20303.6,0.047
3,BTC/USDT,"[20302.8, 0.012]","[20303.8, 0.002]",1662128478476,2022-09-02T14:21:18.476Z,1879833726652,20302.8,0.012,20303.8,0.002
4,BTC/USDT,"[20302.7, 0.262]","[20304.6, 1.328]",1662128478476,2022-09-02T14:21:18.476Z,1879833726652,20302.7,0.262,20304.6,1.328
...,...,...,...,...,...,...,...,...,...,...
495,BTC/USDT,"[20246.1, 0.394]","[20358.3, 0.911]",1662128478476,2022-09-02T14:21:18.476Z,1879833726652,20246.1,0.394,20358.3,0.911
496,BTC/USDT,"[20246.0, 0.645]","[20358.4, 0.213]",1662128478476,2022-09-02T14:21:18.476Z,1879833726652,20246.0,0.645,20358.4,0.213
497,BTC/USDT,"[20245.8, 0.246]","[20358.5, 0.055]",1662128478476,2022-09-02T14:21:18.476Z,1879833726652,20245.8,0.246,20358.5,0.055
498,BTC/USDT,"[20245.7, 0.031]","[20358.6, 6.3]",1662128478476,2022-09-02T14:21:18.476Z,1879833726652,20245.7,0.031,20358.6,6.300


In [37]:
orderbook_df = orderbook_df[bid_ask_columns]
orderbook_df

,timestamp,bid_price,bid_size,ask_price,ask_size
0,1662128478476,20303.2,26.543,20303.3,0.010
1,1662128478476,20303.1,1.820,20303.4,0.001
2,1662128478476,20303.0,2.740,20303.6,0.047
3,1662128478476,20302.8,0.012,20303.8,0.002
4,1662128478476,20302.7,0.262,20304.6,1.328
...,...,...,...,...,...
495,1662128478476,20246.1,0.394,20358.3,0.911
496,1662128478476,20246.0,0.645,20358.4,0.213
497,1662128478476,20245.8,0.246,20358.5,0.055
498,1662128478476,20245.7,0.031,20358.6,6.300


In [40]:
orderbook_df[:10]

,timestamp,bid_price,bid_size,ask_price,ask_size
0,1662128478476,20303.2,26.543,20303.3,0.010
1,1662128478476,20303.1,1.820,20303.4,0.001
2,1662128478476,20303.0,2.740,20303.6,0.047
3,1662128478476,20302.8,0.012,20303.8,0.002
4,1662128478476,20302.7,0.262,20304.6,1.328
5,1662128478476,20302.6,11.522,20304.7,0.973
6,1662128478476,20302.5,3.246,20304.8,1.218
7,1662128478476,20302.0,0.006,20305.2,0.900
8,1662128478476,20301.9,0.002,20305.5,0.123
9,1662128478476,20301.8,0.072,20305.6,0.006


- The top of the book in CCXT is sorted by price
- All columns present in CryptoChassis are present in CCXT output

## `_download_bid_ask` method proposal

In [44]:
def _download_bid_ask(extractor, currency_pair: str, *, depth=10):
    """
    Download bid-ask data from CCXT.

    :param depth: depth of the order book to download.
    """
    # Convert symbol to CCXT format, e.g. "BTC_USDT" -> "BTC/USDT".
    currency_pair = extractor.convert_currency_pair(currency_pair)
    # Download order book data.
    order_book = extractor._exchange.fetch_order_book(currency_pair)
    order_book = pd.DataFrame.from_dict(order_book)
    order_book = order_book.loc[:depth]
    # Separate price and size into columns.
    order_book[["bid_price", "bid_size"]] = pd.DataFrame(
        order_book.bids.to_list(), index=order_book.index
    )
    order_book[["ask_price", "ask_size"]] = pd.DataFrame(
        order_book.asks.to_list(), index=order_book.index
    )
    # Select bid/ask columns.
    bid_ask_columns = [
        "timestamp",
        "bid_price",
        "bid_size",
        "ask_price",
        "ask_size",
    ]
    bid_ask = order_book[bid_ask_columns]
    return bid_ask

In [45]:
data = _download_bid_ask(ccxt_extractor, "BTC_USDT")
data

,timestamp,bid_price,bid_size,ask_price,ask_size
0,1662133361198,20292.0,3.473,20292.1,11.423
1,1662133361198,20291.9,0.255,20292.2,0.001
2,1662133361198,20291.8,0.805,20292.5,0.006
3,1662133361198,20291.7,0.123,20292.6,0.006
4,1662133361198,20291.4,0.123,20292.7,1.018
5,1662133361198,20291.2,0.006,20292.9,0.086
6,1662133361198,20291.1,0.002,20293.0,0.132
7,1662133361198,20290.9,0.151,20293.2,0.003
8,1662133361198,20290.8,0.811,20293.5,0.414
9,1662133361198,20290.7,0.369,20293.6,1.600


## Check universe

Verify that all symbols for which we download bid/ask from CryptoChassis are accessible via CCXT.

In [50]:
binance_universe = imvcounun.get_vendor_universe("crypto_chassis", "download")[
    "binance"
]
binance_universe

['ADA_USDT',
 'BNB_USDT',
 'BTC_USDT',
 'DOGE_USDT',
 'EOS_USDT',
 'ETH_USDT',
 'SOL_USDT']

In [51]:
for symbol in binance_universe:
    try:
        _download_bid_ask(ccxt_extractor, symbol)
        print(f"Successfully downloaded {symbol}.")
    except:
        print(f"Symbol {symbol} failed to download.")

Successfully downloaded ADA_USDT.
Successfully downloaded BNB_USDT.
Successfully downloaded BTC_USDT.
Successfully downloaded DOGE_USDT.
Successfully downloaded EOS_USDT.
Successfully downloaded ETH_USDT.
Successfully downloaded SOL_USDT.
